### Compare units that canceled with DDM-like units for this category

####Output: ccm_category_neurons.mat files

In [ ]:
dataPath = fullfile(projectRoot,'data',projectDate,subject);

cEpochList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccNoPostsacc','presaccPeak'};
dgEpochList = {'Stim','Sacc','Post'};

cEpochList = {'presaccRamp','presaccPeak'};
cEpochList = {'presaccNoPostsacc'};
dgEpochList = {'Stim'};
cancelEpoch = 'presaccNoPostsaccCancel';

for i = 1 : length(cEpochList)
    for j = 1 : length(dgEpochList)
        
        dgEpoch = dgEpochList{j};
        cEpoch = cEpochList{i};




        % LOAD ALL RELEVANT DATA AND LISTS

        % Load Classic neuron classifications
        cFileName = fullfile(dataPath, ['ccm_',cEpochList{i},'_neurons', addMulti]);
        c = load(cFileName);
        classic = c.neurons;

        
        % Load Ding and gold (choice) neuron classifications
        dgFileName = fullfile(dataPath, ['ccm_ddm', dgEpochList{j}, '_neurons', addMulti]);
        dg = load(dgFileName);
        ddm = dg.neurons;


        % Load Cancel neuron classifications 
        caFileName = fullfile(dataPath, ['ccm_',cancelEpoch,'_neurons', addMulti]);
        ca = load(caFileName);
        cancel = ca.neurons;




        [~, iCla, iDdm] = setxor(classic, ddm);
        classicDdm = intersect(classic, ddm);
        classicNoDdm = classic(iCla, :);
        ddmNoClassic = ddm(iDdm, :);


        [~, iCla, iCan] = setxor(classic, cancel);
        classicCancel = intersect(classic, cancel);
        classicNoCancel = classic(iCla, :);
        cancelNoClassic = cancel(iCan, :);


        [~, iCan, iDdm] = setxor(cancel, classicDdm);
        classicDdmCancel = intersect(classicDdm, cancel);
        classicCancelNoDdm = cancel(iCan,:);
        classicDdmNoCancel = classicDdm(iDdm,:);

        classicNoDdmNoCancel = intersect(classicNoDdm, classicNoCancel);
        

        % Save the units as population data, to be called later efficiently
        neurons = classicDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'Ddm',dgEpochList{j},'_neurons', addMulti]), 'neurons')

        neurons = classicNoDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'NoDdm',dgEpochList{j},'_neurons', addMulti]), 'neurons')

        neurons = ddmNoClassic;
        save(fullfile(dataPath, ['ccm_ddm',dgEpochList{j},'No',cEpochList{i},'_neurons', addMulti]), 'neurons')

        neurons = cancelNoClassic;
        save(fullfile(dataPath, ['ccm_cancelNo',cEpochList{i},'_neurons', addMulti]), 'neurons')

        neurons = classicDdmCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'Ddm',dgEpochList{j},'Cancel_neurons', addMulti]), 'neurons')

        neurons = classicCancelNoDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'CancelNoDdm',dgEpochList{j},'_neurons', addMulti]), 'neurons')

        neurons = classicDdmNoCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'Ddm',dgEpochList{j},'NoCancel_neurons', addMulti]), 'neurons')

        neurons = classicNoDdmNoCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'NoDdm',dgEpochList{j},'NoCancel_neurons', addMulti]), 'neurons')


    end
end



### Population spike density functions for each category of neuron

In [ ]:
%% matlab
cEpochList = {'presaccNoPostsacc'};
cEpochList = {'presaccNoVis','presaccRamp','visPresacc'};
cEpochList = {'presaccRamp','presaccPeak'};
cEpochList = {'presacc'};

dgEpochList = {'Stim'};

opt             = ccm_options;
opt.sessionSet  = [];
opt.howProcess  = 'print';
opt.plotFlag    = false;
opt.dataType    = 'neuron';
opt.collapseTarg 	= true;
opt.doStops 	= true;
opt.multiUnit 	= multiUnit;


        
% Classic alone

for i = 1 : length(cEpochList)
    
    fprintf('\tClassic Alone\t%s\n',cEpochList{i})
    load(fullfile(dataPath, ['ccm_',cEpochList{i},'_neurons', addMulti]))

    opt.categoryName = cEpochList{i};
    opt.sessionArray = neurons.sessionID;
    opt.unitArray   = neurons.unit;
    opt.rfList      = neurons.rf;
    opt.hemisphereList      = neurons.hemisphere;


    Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

    save(fullfile(dataPath, ['ccm_',cEpochList{i},'_neuron_population', addMulti]), 'Data')
    clear Data
end    







% Ddm alone
for i = 1 : length(dgEpochList)
    fprintf('\tDDM Alone\tddm%s\n',dgEpochList{i})
    load(fullfile(dataPath, ['ccm_ddm',dgEpochList{i},'_neurons', addMulti]))

    opt.categoryName = ['ddm',dgEpochList{i}];
    opt.sessionArray = neurons.sessionID;
    opt.unitArray   = neurons.unit;
    opt.rfList      = neurons.rf;
    opt.hemisphereList      = neurons.hemisphere;


    Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

    save(fullfile(dataPath, ['ccm_ddm',dgEpochList{i},'_neuron_population', addMulti]), 'Data')
    clear Data
end

    
    
    
    
% Classic w.r.t. canceling

for i = 1 : length(cEpochList)
    categoryList = {[cEpochList{i},'Cancel'], [cEpochList{i},'NoCancel']}

    for j = 1 : length(categoryList)
        fprintf('\tClassic wrt Canceling\t%s\n',categoryList{j})
        load(fullfile(dataPath, ['ccm_',categoryList{j},'_neurons', addMulti]))

        opt.categoryName = categoryList{j};
        opt.sessionArray = neurons.sessionID;
        opt.unitArray   = neurons.unit;
        opt.rfList      = neurons.rf;
        opt.hemisphereList      = neurons.hemisphere;


        Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

        save(fullfile(dataPath, ['ccm_',categoryList{j},'_neuron_population', addMulti]), 'Data')
        clear Data
    end
end
    
    
    
    
    
% Classic w.r.t. DDM    

for i = 1 : length(cEpochList)
    for k = 1 : length(dgEpochList)

    
        % Classic w.r.t. DDM, Classic w.r.t. DDM and Canceling
        categoryList = {[cEpochList{i},'Ddm',dgEpochList{k}], [cEpochList{i},'NoDdm',dgEpochList{k}], ['ddm',dgEpochList{k},'No',cEpochList{i}]}
        for j = 1 : length(categoryList)
            fprintf('\tClassic wrt DDM and canceling\t%s\n',categoryList{j})
            load(fullfile(dataPath, ['ccm_',categoryList{j},'_neurons', addMulti]))

            opt.categoryName = categoryList{j};
            opt.sessionArray = neurons.sessionID;
            opt.unitArray   = neurons.unit;
            opt.rfList      = neurons.rf;
            opt.hemisphereList      = neurons.hemisphere;


            Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

            save(fullfile(dataPath, ['ccm_',categoryList{j},'_neuron_population', addMulti]), 'Data')
            clear Data
        end
    end
end
    


    

% Classic w.r.t. DDM and Canceling    

for i = 1 : length(cEpochList)
    for k = 1 : length(dgEpochList)

    
        % Classic w.r.t. DDM, Classic w.r.t. DDM and Canceling
        categoryList = {[cEpochList{i},'Ddm',dgEpochList{k},'Cancel'], [cEpochList{i},'Cancel','NoDdm',dgEpochList{k}], [cEpochList{i},'Ddm',dgEpochList{k},'NoCancel'], [cEpochList{i},'NoDdm',dgEpochList{k},'NoCancel']}
        for j = 1 : length(categoryList)
            fprintf('\tClassic wrt DDM and canceling\t%s\n',categoryList{j})
            load(fullfile(dataPath, ['ccm_',categoryList{j},'_neurons', addMulti]))

            opt.categoryName = categoryList{j};
            opt.sessionArray = neurons.sessionID;
            opt.unitArray   = neurons.unit;
            opt.rfList      = neurons.rf;
            opt.hemisphereList      = neurons.hemisphere;


            Data = ccm_population_neuron(subject,projectRoot,projectDate, opt)

            save(fullfile(dataPath, ['ccm_',categoryList{j},'_neuron_population', addMulti]), 'Data')
            clear Data
        end
    end
end
    


    
 



In [ ]:
subject = 'broca';
categoryList = {'presaccCancel'};
%categoryList = {'presaccDdm'};
%categoryList = {'presaccCancelNoDdm'};
%categoryList = {'presaccDdmNoCancel'};
categoryList = {'presaccRampDdmStimCancel'};
%categoryList = {'presaccNoDdm'}
categoryList = {'presaccDdm'}
categoryList = {'ddmNoPresacc', 'presaccDdm'}
categoryList = {'presaccNoDdm', 'presaccNoCancel'}
categoryList = {'presaccNoCancel'}
opt = ccm_population_neuron_plot;

opt.doStops = true;
opt.easyOnly = true;
opt.inOnly = false;
opt.multiUnit = multiUnit;
opt.normalize = normalizeData;

for i = 1 : length(categoryList)
    
    opt.categoryName = categoryList{i};

    ccm_population_neuron_plot(subject,projectRoot,projectDate,opt);
end


#Continue to Summary Analyses Notebook